In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utils.functions import img2real
import json
import pickle
import numpy as np

In [2]:
df = pd.read_csv('data/train_data.csv')
df = df.dropna()
df['error'] = np.sqrt((df['x']-df['xc'])**2 + (df['y']-df['yc'])**2)

min_max_scalers = {}

for column in df.columns:
    if column in  ['xc', 'yc', 'error']:
        min_max_scalers[column] = MinMaxScaler() #(df[column].min(), df[column].max())
        df[column] = min_max_scalers[column].fit_transform(df[[column]])
        with open('scalers/'+column+'.pkl', 'wb') as f:
            pickle.dump(min_max_scalers[column], f)

#with open('data/min_max_scalers.json', 'w') as f:
#    json.dump(min_max_scalers, f)

df.to_csv(f"data/train_data_clean.csv", index=False) 

df.head()

,x,y,z,t0,t1,t2,t3,xc_px,yc_px,xc,yc,diagonal,error
7,0.16,-0.19,0.12,30.100908,33.948155,75.041009,41.092854,145.0,-275.0,0.591045,0.987826,73.389373,1.000000
8,0.17,-0.18,0.12,33.363423,34.952635,76.655631,41.702995,160.0,-264.0,0.635821,0.968696,72.801099,0.988214
9,0.18,-0.17,0.12,36.636577,34.952635,76.655631,41.702995,178.0,-249.0,0.689552,0.942609,72.780492,0.974219
10,0.19,-0.16,0.12,39.899092,33.948155,75.041009,41.092854,191.0,-235.0,0.728358,0.918261,72.835431,0.958709
11,0.20,-0.15,0.12,43.130102,31.753588,71.469230,39.715641,208.0,-222.0,0.779104,0.895652,75.239617,0.957535


In [3]:
pos_inicial = img2real((314,134),27) # pos_inicial e cameraHeight utilizado
print(pos_inicial)

error_start = np.sqrt((0.035-pos_inicial[0])**2 + (0-pos_inicial[1])**2)
print(error_start)

pos_inicial = np.array([
    min_max_scalers['xc'].transform(np.array(pos_inicial[0]).reshape(1, -1))[0][0],
    min_max_scalers['yc'].transform(np.array(pos_inicial[1]).reshape(1, -1))[0][0],
    min_max_scalers['error'].transform(np.array(error_start).reshape(1, -1))[0][0]
])

print(pos_inicial)

(-5.652, 2.412)
6.177354854628314
[-0.77910448  0.7426087   1.08560483]


c:\Users\pedro\Documents\NOVOdeltadelta\pog\mlp-robot\.auto\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\pedro\Documents\NOVOdeltadelta\pog\mlp-robot\.auto\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\pedro\Documents\NOVOdeltadelta\pog\mlp-robot\.auto\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [4]:
def perc_pos(value, perc):
    return int(value*perc)/100

pos_inicial = pos_inicial.reshape(1,-1)
input_data = torch.Tensor()
output_data = torch.Tensor()

for i in range(len(df)):
     row = df.iloc[i]
     x_fin, y_fin = row['x']*100, row['y']*100
     pos_hist = torch.Tensor(pos_inicial)
     for p in [0.25, 0.5, 0.75]:
          check=True
          while(check):
               path_x = perc_pos(x_fin,p)
               path_y = perc_pos(y_fin,p)
               pos = df.loc[(df.x==path_x)&(df.y==path_y),['xc','yc','error']].values
               if(pos.size==0):
                    p += 0.05
               else:
                    check = False
          pos_hist = torch.cat((pos_hist, torch.Tensor(pos)))
     input_data = torch.cat((input_data, pos_hist.unsqueeze(0)))
     output_data = torch.cat((output_data, torch.Tensor([row[['t0','t1','t2','t3']].values])))

input_data
     

C:\Users\pedro\AppData\Local\Temp\ipykernel_10376\1405297032.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  output_data = torch.cat((output_data, torch.Tensor([row[['t0','t1','t2','t3']].values])))


tensor([[[-0.7791,  0.7426,  1.0856],
         [ 0.1134,  0.6122,  0.1981],
         [ 0.2119,  0.6817,  0.3258],
         [ 0.4060,  0.8383,  0.6644]],

        [[-0.7791,  0.7426,  1.0856],
         [ 0.1194,  0.5878,  0.1519],
         [ 0.2119,  0.6817,  0.3258],
         [ 0.4090,  0.8087,  0.6138]],

        [[-0.7791,  0.7426,  1.0856],
         [ 0.1194,  0.5878,  0.1519],
         [ 0.2687,  0.6713,  0.3185],
         [ 0.4567,  0.7826,  0.5913]],

        ...,

        [[-0.7791,  0.7426,  1.0856],
         [ 0.1015,  0.4400,  0.1445],
         [ 0.1015,  0.4400,  0.1445],
         [ 0.1015,  0.4400,  0.1445]],

        [[-0.7791,  0.7426,  1.0856],
         [ 0.1254,  0.4678,  0.0843],
         [ 0.1254,  0.4678,  0.0843],
         [ 0.1254,  0.4678,  0.0843]],

        [[-0.7791,  0.7426,  1.0856],
         [ 0.1403,  0.4765,  0.0614],
         [ 0.1403,  0.4765,  0.0614],
         [ 0.1403,  0.4765,  0.0614]]])

In [5]:
output_data

tensor([[ 30.1009,  33.9482,  75.0410,  41.0929],
        [ 33.3634,  34.9526,  76.6556,  41.7030],
        [ 36.6366,  34.9526,  76.6556,  41.7030],
        ...,
        [125.0000, 146.0643, 172.2480,  26.1836],
        [110.9638, 152.7918, 172.6161,  19.8243],
        [106.5650, 148.0869, 172.4426,  24.3557]])

In [7]:
BATCH_SIZE = 8

X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=42, shuffle=True)

X_test = X_test[4:,:,:]
y_test = y_test[4:,:] # p/ evitar data leakage

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE)

# Separação em conj de validação
#X_train, y_train, X_val, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42, shuffle=True)

#train_loader

In [14]:
# Dados de exemplo
# Suponha batch_size = 1, seq_len = 4, input_size = 2 (x, y)
from itertools import product
from utils.rnn import ControlRNN, train_net
import torch
import torch.nn as nn

# Inicializa a RNN
model = ControlRNN(input_size=4)
criterion = nn.MSELoss()

hidden_size = [300, 100, 50]
num_layers = [1,2]
dropout_rate = [0,0.25,0.75]
learning_rate = [0.0001, 0.001, 0.01]
weight_decay = [0, 0.0005]
activation = ['relu', 'tanh']
optimizer = ['adam', 'sgd']
best_model = None
best_score = None

params_grid = product(hidden_size, num_layers, dropout_rate, learning_rate,
                      activation, optimizer, weight_decay)

count = 1
for hs, nl, dr, lr, a, o, wd in params_grid:
    if(not((dr>0) and (nl==1))):
        model = ControlRNN(input_size=3, hidden_size=hs, 
                        num_layers=nl,
                        activation=a, dropout_rate=dr)
        if(o=='adam'):
            opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        else:
            opt = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd)
        
        
        model, _, scores = train_net(model, train_loader, test_loader, 25, opt, verbose=0)
        score = np.mean(scores[1])
        print(f'[{count}] Val-MSE: {score:.3f}')
        if(best_score is None or best_score > score):
            print('[Melhor modelo atualizado]')
            print('hs:{}, nl:{}, dr:{}, lr:{}, a:{}, o:{}, wd:{}'.format(hs, nl, dr, lr, a, o, wd))
            best_model = model
            best_score = score
        count += 1
                                

# Teste
predicted = best_model(input_data).detach()
print("Predição:", predicted)

[1] Val-MSE: 32.721
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.0001, a:relu, o:adam, wd:0
[2] Val-MSE: 32.682
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.0001, a:relu, o:adam, wd:0.0005
[3] Val-MSE: 31.027
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.0001, a:relu, o:sgd, wd:0
[4] Val-MSE: 33.874
[5] Val-MSE: 68.404
[6] Val-MSE: 68.613
[7] Val-MSE: 30.690
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.0001, a:tanh, o:sgd, wd:0
[8] Val-MSE: 30.674
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.0001, a:tanh, o:sgd, wd:0.0005
[9] Val-MSE: 28.743
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.001, a:relu, o:adam, wd:0
[10] Val-MSE: 28.708
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.001, a:relu, o:adam, wd:0.0005
[11] Val-MSE: nan
[12] Val-MSE: nan
[13] Val-MSE: 33.547
[14] Val-MSE: 33.636
[15] Val-MSE: 28.989
[16] Val-MSE: 29.004
[17] Val-MSE: 23.636
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.01, a:relu, o:adam, wd:0
[18] Val-MSE: 26.7

In [29]:
predicted = best_model(X_test).detach()

score = ((predicted - y_test)**2).mean()

print(f'''MSE: {score:.4f}
RMSE: {score**0.5:.4f}''')

MSE: 40.5490
RMSE: 6.3678


In [31]:
best_model

ControlRNN(
  (rnn): RNN(3, 300, num_layers=2, batch_first=True)
  (fc): Linear(in_features=300, out_features=4, bias=True)
)

Melhor modelo:

- Número de camadas recorrentes: 2
- Número de neurônios ocultos: 300
- Funcção de ativação: ReLU
- Taxa de dropout: 0
- Ottimizador: Adam
- Learning rate: 0.01
- Weight decay: 0.0005

In [30]:
#with open(, 'wb') as f:
best_model.eval()
torch.save(best_model.state_dict(), 'models/nn_model.pt')